In [1]:
import csv

In [2]:
from bs4 import BeautifulSoup

In [3]:
import requests as uReq

In [4]:
html_text = uReq.get('https://www.dsebd.org/latest_share_price_scroll_l.php').text

In [5]:
soup = BeautifulSoup(html_text,'lxml')

In [6]:
stocks_tcode_url = soup.find('table', class_ = 'table table-bordered background-white shares-table fixedHeader').find_all('a')

In [75]:
list_temp_data_hold = list()

In [7]:
stock_tcode_url_dict = dict()

In [8]:
for stock in stocks_tcode_url:
    stock_tcode_url_dict[f'{stock.text.strip()}'] = 'https://www.dsebd.org/' + stock['href']

In [78]:
for stock_tcode, url in sorted(stock_tcode_url_dict.items()):
    
    #browse and download from the website, store the extracted lxml parsed page on soup_company.
    html_text_company = uReq.get(url).text
    soup_company = BeautifulSoup(html_text_company,'lxml')
    
    #group all the table from stock's page and extract disclosures 
    company_table_data = soup_company.find_all('table', class_ = 'table table-bordered background-white')
    sector = company_table_data[1].find_all('tr', class_='alt')[1].find_all('td')[1].text #sector
    pe_quarter = company_table_data[4].find('tr', class_ = 'alt').find_all('td')[-1].text #P/E: Quarterly report
    pe_annual = company_table_data[5].find('tr', class_='alt').find_all('td')[-1].text #P/E audited report
    
    trading_price = company_table_data[0].find('tr').find('td').text #last trading Price
    recent_dividend = company_table_data[7].find_all('tr')[-1].find_all('td')[-2].text.strip() #dividend return
    
    navps = company_table_data[6].find_all('tr')[-1].find_all('td')[-6].text.strip() #NAVPS
    audit_year = company_table_data[7].find_all('tr')[-1].td.text #year of last audited financial report
    market_category = company_table_data[9].find('tr').find_next_sibling('tr').find('td').find_next_sibling('td').text.strip() #category
    list_temp_data_hold.append([stock_tcode, trading_price, sector, market_category, recent_dividend, pe_quarter, pe_annual, navps, audit_year])

In [230]:
len(stock_tcode_url_dict)

383

In [80]:
for stock in list_temp_data_hold:
    print(stock)

['1JANATAMF', '6.50', 'Mutual Funds', 'A', '13.00', '8.13', '2.56', '11.94', '2021']
['1STPRIMFMF', '20.00', 'Mutual Funds', 'A', '8.00', '13.38', '52.11', '11.62', '2020']
['AAMRANET', '45.90', 'IT Sector', 'A', '5.00, 5% B', '22.5', '21.45', '37.09', '2021']
['AAMRATECH', '32.20', 'IT Sector', 'A', '5.00, 5% B', '15.19', '22.21', '23.93', '2021']
['ABB1STMF', '5.70', 'Mutual Funds', 'A', '8.00', '8.38', '2.18', '11.97', '2021']
['ABBANK', '13.60', 'Bank', 'B', '5% B', '23.18', '27.2', '32.26', '2020']
['ACFL', '31.50', 'Textile', 'A', '11.00', '29.17', '25.2', '34.72', '2021']
['ACI', '294.60', 'Pharmaceuticals & Chemicals', 'A', '65%, 15%B', '18.74', '53.56', '162.65', '2021']
['ACIFORMULA', '166.70', 'Pharmaceuticals & Chemicals', 'A', '30.00, 5% B', '35.62', '35.17', '66.77', '2021']
['ACMELAB', '92.00', 'Pharmaceuticals & Chemicals', 'A', '25.00', '8.3', '12.4', '95.04', '2021']
['ACMEPL', '31.20', 'Pharmaceuticals & Chemicals', 'N', '-', '16.6', '18.91', '18.48', '2021']
['ACTIV

list_temp_data_hold.clear()

In [176]:
html_text_company = uReq.get(stock_tcode_url_dict['TALLUSPIN']).text

In [177]:
soup_company = BeautifulSoup(html_text_company,'lxml')

In [178]:
company_table_data = soup_company.find_all('table', class_ = 'table table-bordered background-white')

In [179]:
if '-' not in company_table_data[0].find('tr').find('td').text:
    trading_price = company_table_data[0].find('tr').find('td').text.replace(',','') #last trading Price
elif '-' in company_table_data[0].find('tr').find('td').text:
    trading_price = company_table_data[0].find('tr').find('td').text

In [180]:
trading_price

'10.70'

In [181]:
'-' in company_table_data[0].find('tr').find('td').text

False

In [182]:
company_table_data[1].find_all('tr', class_='alt')[1].find_all('td')[1].text #sector

'Textile'

In [183]:
sector = company_table_data[1].find_all('tr', class_='alt')[1].find_all('td')[1].text #sector

In [184]:
company_table_data[4].find('tr', class_ = 'alt').find_all('td')[-1].text #P/E: Quarterly report

'-'

In [185]:
pe_quarter = company_table_data[4].find('tr', class_ = 'alt').find_all('td')[-1].text #P/E: Quarterly report

In [186]:
company_table_data[5].find('tr', class_='alt').find_all('td')[-1].text.strip() #P/E audited report

'n/a'

In [187]:
pe_annual = company_table_data[5].find('tr', class_='alt').find_all('td')[-1].text.strip() #P/E audited report

In [188]:
pe_annual

'n/a'

In [189]:
dividend_check = company_table_data[7].find_all('tr')[-1].find_all('td')[-2].text.strip()

In [190]:
len(dividend_check.split(' '))

1

In [191]:
dividend_check

'-'

In [192]:
dividend_return(dividend_check, trading_price)

'-'

In [193]:
def dividend_return(dividend_check, trading_price):
    
    if ',' in dividend_check or len(dividend_check.split(' ')) == 3:
        dividends = dividend_check.split(',') if ',' in dividend_check else dividend_check.split(' ', 1)   #stores dividend paid in both cash and bonus shares
        
        dividend = dividend_in_bonus_cash(dividends, trading_price)
    
    elif '%' in dividend_check and ',' not in dividend_check: 
        dividend = dividend_in_bonus(dividend_check, trading_price)
    
    elif '-' in dividend_check:
        dividend = dividend_check   #when dividend isn't mentioned on the website
    
    elif 'B' not in dividend_check and '%' not in dividend_check and '-' not in dividend_check and ',' not in dividend_check:
        dividend = dividend_cash(dividend_check)   #Dividend paid in cash
    
    return dividend

In [194]:
with open('accounts.csv', mode = 'w', newline = '') as accounts:
    writer = csv.writer(accounts)
    for stock in list_temp_data_hold:
        writer.writerow(stock)

NameError: name 'list_temp_data_hold' is not defined

In [195]:
def dividend_in_bonus_cash(cash_and_bonus_dividends, market_price):
    """This function calculates total dividend from cash and bonus issues"""
    
    for dividend in cash_and_bonus_dividends:
        if 'B' and '%' not in dividend:
            cash_dividend = float(dividend)
            
        elif 'B' in dividend:
            bonus_dividend = float(dividend.strip().replace('%', '').replace('%','').replace('B',''))
    
    
    if '-' not in market_price:
        return round(((cash_dividend / 100) * 10) + ((bonus_dividend / 100) * float(market_price)), 2)
    
    elif '-' in market_price:
        return [cash_dividend, bonus_dividend]

In [16]:
def dividend_in_bonus(bonus_dividend, market_price):
    '''Calculates the amount of last bonus share dividend'''
    dividend = float(bonus_dividend.strip().replace('%', '').replace('%','').replace('B',''))
    
    if '-' not in market_price:
        return round(((dividend / 100) * float(market_price)), 2)
    
    elif '-' in market_price:
        return str(dividend) + '%B'

In [17]:
dividend_in_bonus('12.12', '10')

1.21

In [197]:
def dividend_cash(cash_dividend):
    return round((float(cash_dividend) / 100) * 10 , 2)

In [234]:
def calculate_navps(navps):
    if '-' in navps or 'n/a' in navps:
        return navps
    elif '-' not in navps:
        return float(navps.replace(',',''))

In [235]:
calculate_navps('12.11')

12.11

In [220]:
pe_quarter

'-'

In [221]:
pe_annual

'n/a'

In [222]:
'-' in pe_quarter or 'n/a' in pe_quarter

True

In [201]:
'-' in pe_annual or 'n/a' in pe_annual

True

In [208]:
def calculate_eps(pe, stock_price):
    '''Receives P/E and Trading price as arguments and returns eps'''
    if '-' in pe or 'n/a' in pe or '-' in stock_price:
        eps = None
    else:
        eps = round(float(stock_price) / float(pe), 2)
    
    return eps

In [209]:
'-' not in trading_price

True

In [210]:
calculate_eps(pe_annual, trading_price)

In [38]:
company_table_data[7].find_all('tr')[-1].find_all('td')[-2].text.strip()

'30.00'

In [39]:
company_table_data[7].find_all('tr')[-1].find_all('td')[-2].text.strip() #dividend return

'30.00'

In [72]:
company_table_data[9].find('tr').find_next_sibling('tr').find('td').find_next_sibling('td').text.strip() #category

'A'

In [227]:
navps = company_table_data[6].find_all('tr')[-1].find_all('td')[-6].text.strip() #NAVPS

In [228]:
navps

'11.97'

In [42]:
company_table_data[7].find_all('tr')[-1].td.text #audit year of financial report

'2020'

In [43]:
import re

In [ ]:
re.findall('\d*\.?\d+', '12.22')

In [156]:
',' in 'mmhjg,' and '-' in 'hgjhdjhv,'

False

In [171]:
'B' not in 'ABC' or '%' not in 'ABC'

True

In [7]:
a_number = 12

In [13]:
def fun(number):
    return 10

In [14]:
a_number = fun(a_number)

In [15]:
a_number

10